# Probabilistic Wind Power Prediction Results

This notebook evaluates probabilistic predictions with uncertainty quantification using quantile regression.


In [ ]:
import sys
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import torch
import torch.nn as nn

project_root = Path().resolve().parent

# Ensure results directories exist
(project_root / 'results' / 'figures').mkdir(parents=True, exist_ok=True)
(project_root / 'results' / 'metrics').mkdir(parents=True, exist_ok=True)
(project_root / 'data' / 'processed').mkdir(parents=True, exist_ok=True)
sys.path.insert(0, str(project_root / 'src'))

from preprocessing import time_aware_split, prepare_sequences, FeatureScaler
from models.transformer import TimeSeriesTransformer
from models.probabilistic_head import QuantileRegressionHead, quantile_loss
from training import set_seed, train_model
from evaluation import (
    compute_metrics,
    evaluate_probabilistic,
    plot_predictions,
    plot_uncertainty,
    plot_error_vs_wind_speed
)
from torch.utils.data import DataLoader, Dataset

set_seed(42)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

plt.style.use('seaborn-v0_8-darkgrid')
%matplotlib inline


## Create Probabilistic Model


In [ ]:
# Load cleaned data
# Check for both compressed and uncompressed versions
data_path_csv = project_root / 'data' / 'processed' / 'scada_cleaned.csv'
data_path_gz = project_root / 'data' / 'processed' / 'scada_cleaned.csv.gz'
mapping_path = project_root / 'data' / 'processed' / 'feature_mapping.json'

# Determine which file exists
if data_path_gz.exists():
    print(f"Loading compressed data from: {data_path_gz}")
    df = pd.read_csv(data_path_gz, index_col=0, parse_dates=True, compression='gzip')
elif data_path_csv.exists():
    print(f"Loading data from: {data_path_csv}")
    df = pd.read_csv(data_path_csv, index_col=0, parse_dates=True)
else:
    raise FileNotFoundError(
        f"Cleaned data file not found!\n"
        f"Expected one of:\n"
        f"  - {data_path_csv}\n"
        f"  - {data_path_gz}\n\n"
        f"Please run notebook 01_data_exploration.ipynb first to generate the cleaned data."
    )

# Load feature mapping
if not mapping_path.exists():
    raise FileNotFoundError(
        f"Feature mapping file not found: {mapping_path}\n"
        f"Please run notebook 01_data_exploration.ipynb first."
    )

with open(mapping_path, 'r') as f:
    feature_mapping = json.load(f)
target_col = feature_mapping['target']
feature_cols = feature_mapping['features']
ws_col = feature_mapping['all_features'].get('wind_speed')

# Split and prepare
train_df, val_df, test_df = time_aware_split(df, train_ratio=0.7, val_ratio=0.15, test_ratio=0.15)
scaler = FeatureScaler(method='standard')
X_train_scaled = scaler.fit_transform(train_df[feature_cols])
X_val_scaled = scaler.transform(val_df[feature_cols])
X_test_scaled = scaler.transform(test_df[feature_cols])

y_train = train_df[target_col].values
y_val = val_df[target_col].values
y_test = test_df[target_col].values

# Prepare sequences with memory limits
sequence_length = 48
max_train_samples = 500_000  # Limit to avoid memory issues
max_val_samples = 100_000
max_test_samples = 100_000

print(f"Preparing sequences with length {sequence_length}...")
print(f"Limiting to {max_train_samples:,} training sequences")

# Convert to float32 to save memory
X_train_f32 = X_train_scaled.values.astype(np.float32)
y_train_f32 = y_train.astype(np.float32)
X_val_f32 = X_val_scaled.values.astype(np.float32)
y_val_f32 = y_val.astype(np.float32)
X_test_f32 = X_test_scaled.values.astype(np.float32)
y_test_f32 = y_test.astype(np.float32)

X_train_seq, y_train_seq = prepare_sequences(
    X_train_f32, y_train_f32, sequence_length,
    max_samples=max_train_samples, dtype=np.float32
)
X_val_seq, y_val_seq = prepare_sequences(
    X_val_f32, y_val_f32, sequence_length,
    max_samples=max_val_samples, dtype=np.float32
)
X_test_seq, y_test_seq = prepare_sequences(
    X_test_f32, y_test_f32, sequence_length,
    max_samples=max_test_samples, dtype=np.float32
)

print(f"\nTrain sequences: {X_train_seq.shape}")
print(f"Val sequences: {X_val_seq.shape}")
print(f"Test sequences: {X_test_seq.shape}")

# Create simple dataset wrapper for pre-prepared sequences
class PreSequenceDataset(Dataset):
    """Dataset wrapper for pre-prepared sequences."""
    def __init__(self, X_seq, y_seq):
        self.X_seq = X_seq
        self.y_seq = y_seq
    
    def __len__(self):
        return len(self.X_seq)
    
    def __getitem__(self, idx):
        return torch.FloatTensor(self.X_seq[idx]), torch.FloatTensor([self.y_seq[idx]] if self.y_seq.ndim == 1 else self.y_seq[idx])

# Create datasets using pre-prepared sequences (to respect max_samples limits)
train_dataset = PreSequenceDataset(X_train_seq, y_train_seq)
val_dataset = PreSequenceDataset(X_val_seq, y_val_seq)
test_dataset = PreSequenceDataset(X_test_seq, y_test_seq)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Create model with probabilistic head
backbone = TimeSeriesTransformer(
    input_size=X_train_scaled.shape[1],
    d_model=128,
    nhead=8,
    num_layers=4,
    dim_feedforward=512,
    patch_size=4,
    dropout=0.1,
    output_size=128  # Output embedding for probabilistic head
).to(device)

quantiles = [0.1, 0.5, 0.9]
prob_head = QuantileRegressionHead(128, quantiles=quantiles).to(device)

# Combined model
class ProbabilisticTransformer(nn.Module):
    def __init__(self, backbone, head):
        super().__init__()
        self.backbone = backbone
        self.head = head
    
    def forward(self, x):
        features = self.backbone(x)
        return self.head(features)

model = ProbabilisticTransformer(backbone, prob_head).to(device)
print(f"Model created with {sum(p.numel() for p in model.parameters()):,} parameters")

## Train Probabilistic Model


In [ ]:
# Training loop with quantile loss
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)
n_epochs = 100
best_val_loss = float('inf')
patience = 15
patience_counter = 0

train_losses = []
val_losses = []

for epoch in range(n_epochs):
    # Training
    model.train()
    train_loss = 0.0
    for batch_X, batch_y in train_loader:
        batch_X = batch_X.to(device)
        batch_y = batch_y.to(device)
        
        optimizer.zero_grad()
        quantile_preds = model(batch_X)
        loss = quantile_loss(quantile_preds, batch_y, quantiles)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        train_loss += loss.item()
    
    # Validation
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for batch_X, batch_y in val_loader:
            batch_X = batch_X.to(device)
            batch_y = batch_y.to(device)
            quantile_preds = model(batch_X)
            loss = quantile_loss(quantile_preds, batch_y, quantiles)
            val_loss += loss.item()
    
    train_loss /= len(train_loader)
    val_loss /= len(val_loader)
    train_losses.append(train_loss)
    val_losses.append(val_loss)
    
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        patience_counter = 0
        torch.save(model.state_dict(), project_root / 'results' / 'checkpoints' / 'probabilistic_model.pt')
    else:
        patience_counter += 1
    
    if (epoch + 1) % 10 == 0:
        print(f"Epoch {epoch+1}/{n_epochs} - Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")
    
    if patience_counter >= patience:
        print(f"Early stopping at epoch {epoch+1}")
        break

# Plot training history
plt.figure(figsize=(12, 5))
plt.plot(train_losses, label='Train Loss')
plt.plot(val_losses, label='Val Loss')
plt.xlabel('Epoch')
plt.ylabel('Quantile Loss')
plt.title('Probabilistic Model Training History')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
figures_dir = project_root / 'results' / 'figures'
figures_dir.mkdir(parents=True, exist_ok=True)
plt.savefig(figures_dir / 'probabilistic_training_history.png', dpi=150)
plt.show()


In [ ]:
# Load best model
model.load_state_dict(torch.load(project_root / 'results' / 'checkpoints' / 'probabilistic_model.pt'))

# Evaluate on test set
model.eval()
quantile_preds_list = []
y_test_list = []

with torch.no_grad():
    for batch_X, batch_y in test_loader:
        batch_X = batch_X.to(device)
        quantile_preds = model(batch_X)
        quantile_preds_list.append(quantile_preds.cpu().numpy())
        y_test_list.append(batch_y.numpy())

quantile_preds_test = np.vstack(quantile_preds_list)
y_test_flat = np.concatenate(y_test_list)

# Get median prediction
median_pred = prob_head.get_median(torch.FloatTensor(quantile_preds_test)).numpy()

# Evaluate
prob_metrics = evaluate_probabilistic(y_test_flat, quantile_preds_test, quantiles)
print("\nProbabilistic Model Metrics:")
for metric, value in prob_metrics.items():
    print(f"  {metric}: {value:.4f}")

# Standard metrics on median
standard_metrics = compute_metrics(y_test_flat, median_pred)
print("\nStandard Metrics (Median Prediction):")
for metric, value in standard_metrics.items():
    print(f"  {metric}: {value:.4f}")


## Visualizations


In [ ]:
# Get uncertainty bounds
lower, upper = prob_head.get_uncertainty_bounds(
    torch.FloatTensor(quantile_preds_test),
    lower_quantile=0.1,
    upper_quantile=0.9
)
lower = lower.numpy()
upper = upper.numpy()

# Plot predictions with uncertainty
plot_uncertainty(
    y_test_flat,
    median_pred,
    lower,
    upper,
    save_path=project_root / 'results' / 'figures' / 'probabilistic_predictions.png',
    title='Probabilistic Predictions with 80% Uncertainty Bounds',
    sample_size=500
)

# Plot predictions vs ground truth
plot_predictions(
    y_test_flat,
    median_pred,
    save_path=project_root / 'results' / 'figures' / 'probabilistic_scatter.png',
    title='Probabilistic Model: Predictions vs Ground Truth'
)

# Error vs wind speed
if ws_col and ws_col in test_df.columns:
    ws_test = test_df[ws_col].values[:len(y_test_flat)]
    plot_error_vs_wind_speed(
        y_test_flat,
        median_pred,
        ws_test,
        save_path=project_root / 'results' / 'figures' / 'error_vs_wind_speed.png',
        title='Prediction Error vs Wind Speed'
    )

# Save results
results_df = pd.DataFrame({
    'true': y_test_flat,
    'median': median_pred,
    'lower_10': lower,
    'upper_90': upper
})
results_df.to_csv(project_root / 'results' / 'metrics' / 'probabilistic_results.csv', index=False)
print("\n✅ Results saved")
